In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import re
from string import punctuation

# Import NLTK packages - natural language processing
import nltk
from nltk.tokenize import word_tokenize
#from nltk.stem import WordNetLemmatizer
#lemmatizer = WordNetLemmatizer()
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

# Import scikit learn - machine learning
from sklearn.preprocessing import MinMaxScaler
scale = MinMaxScaler()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/fernando/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
training = pd.read_csv('train.csv')
training.head()
test = pd.read_csv('test.csv')
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [3]:
training['train_test'] = 1
test['train_test'] = 0
test['target'] = np.NaN
all_data = pd.concat([training, test], sort=True)

In [4]:
# Import NLTK packages - natural language processing
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
#from nltk.stem import WordNetLemmatizer
#lemmatizer = WordNetLemmatizer()
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to /home/fernando/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# Define dictionary of contractions
contractions = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

In [6]:
# Write function to pre-process text
def pre_processing(text):
    
    # Convert to lowercase
    text = text.lower()
    
    # Replace contractions
    for word in text.split():
        if word in contractions:
            text = text.replace(word,contractions[word.lower()])
            
    # Use regex to remove URLs
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE)
    
    # Use regex to remove special characters
    text = re.sub("[^a-zA-Z]"," ",text)
    
    tokens = word_tokenize(text)
    
    # Use stemmer to obtain root words
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    single_string = ' '.join(stemmed_tokens)
    return single_string

all_data['text_clean'] = all_data['text'].apply(lambda x:pre_processing(x))

In [7]:

def score(text):
    score = SentimentIntensityAnalyzer().polarity_scores(text)
    neg= score['neg']
    neu= score['neu']
    pos= score['pos']
    compound=score['compound']
    return [neg,neu,pos,compound]

In [8]:
all_data['score']= all_data['text_clean'].agg(score)


In [9]:
all_data['pos']= all_data.apply(lambda x: x['score'][2],axis=1)
all_data['compound']= all_data.apply(lambda x: x['score'][3],axis=1)
all_data['neg']= all_data.apply(lambda x: x['score'][0],axis=1)
all_data['neu']= all_data.apply(lambda x: x['score'][1],axis=1)
del all_data['score']

In [10]:
all_data['word_count'] = all_data['text'].apply(lambda x:len(str(x).split()))
all_data['unique_word_count'] = all_data['text_clean'].apply(lambda x:len(set(str(x).split())))
all_data['mean_word_length'] = all_data['text'].apply(lambda x: np.mean([len(word) for word in str(x).split()]))
all_data['character_count'] = all_data['text'].apply(lambda x:len(str(x)))
all_data['digit_count'] = all_data['text'].apply(lambda x: np.sum([len(char) for char in x if char.isdigit() == True]))
all_data['non_word_char_count'] = all_data['text'].apply(lambda x: len(re.findall('r\W',x)))
all_data['number_of_URLs'] = all_data['text'].apply(lambda x:len(re.findall(r'http\S+|www\.\S+',x)))
all_data['number_of_hashtags'] = all_data['text'].apply(lambda x: len(re.findall('#',x)))
all_data['number_of_mentions'] = all_data['text'].apply(lambda x: len(re.findall('@',x)))

In [11]:

all_data_scaled = all_data.copy()
all_data_scaled[['word_count','unique_word_count','mean_word_length','character_count','digit_count','non_word_char_count','number_of_hashtags','number_of_URLs','number_of_mentions']] = scale.fit_transform(all_data_scaled[['word_count','unique_word_count','mean_word_length','character_count','digit_count','non_word_char_count','number_of_hashtags','number_of_URLs','number_of_mentions']])



In [12]:
all_data_scaled.keyword = all_data_scaled.keyword.fillna('no_keyword')
all_data_dummies = pd.get_dummies(all_data_scaled,columns=['keyword'])

In [13]:
!pip install xgboost

In [14]:
import xgboost as xgb

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
data = all_data_dummies.select_dtypes(exclude=['object'])

In [17]:
localtrain= data.loc[data['train_test'] == 1]
competitionTest=data.loc[data['train_test'] == 0]

In [18]:
localtrain.head()

,id,target,train_test,pos,compound,neg,neu,word_count,unique_word_count,mean_word_length,...,keyword_weapons,keyword_whirlwind,keyword_wild%20fires,keyword_wildfire,keyword_windstorm,keyword_wounded,keyword_wounds,keyword_wreck,keyword_wreckage,keyword_wrecked
0,1,1.0,1,0.0,0.000,0.000,1.000,0.400000,0.428571,0.124953,...,0,0,0,0,0,0,0,0,0,0
1,4,1.0,1,0.0,-0.340,0.286,0.714,0.200000,0.214286,0.135889,...,0,0,0,0,0,0,0,0,0,0
2,5,1.0,1,0.0,-0.296,0.095,0.905,0.700000,0.607143,0.166297,...,0,0,0,0,0,0,0,0,0,0
3,6,1.0,1,0.0,0.000,0.000,1.000,0.233333,0.214286,0.285366,...,0,0,0,0,0,0,0,0,0,0
4,7,1.0,1,0.0,0.000,0.000,1.000,0.500000,0.500000,0.131707,...,0,0,0,0,0,0,0,0,0,0


In [19]:
y=localtrain['target']
x = localtrain
del x['train_test']
del x['target']
yComp=competitionTest['target']
xComp=competitionTest
del xComp['train_test']
del xComp['target']

In [20]:
y.head()

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: target, dtype: float64

In [21]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=123)

In [22]:
xgb_reg = xgb.XGBRegressor(objective='binary:logistic'\
                           ,colsample_bytree=0.6,learning_rate=0.2,\
                           max_depth=8,alpha=5,n_estimators=200)
xgb_reg.fit(x_train,y_train)
preds=xgb_reg.predict(x_test)
preds1 = np.around(preds)
preds1 = preds1.astype(int)

In [23]:
error= y_test-preds1
tablaErrores=pd.DataFrame(error,columns=['target'])
tablaErrores['error1']=tablaErrores['target']
tablaErrores['error2']= y_test+preds1
tablaErrores.loc[tablaErrores['error1']==-1,'FP']=1
tablaErrores.loc[tablaErrores['error1']!=-1,'FP']=0
tablaErrores.loc[tablaErrores['error2']==2,'TP']=1
tablaErrores.loc[tablaErrores['error2']!=2,'TP']=0
tablaErrores.loc[tablaErrores['error1']==1,'FN']=1
tablaErrores.loc[tablaErrores['error1']!=1,'FN']=0
TP=tablaErrores['TP'].sum()
FP=tablaErrores['FP'].sum()
FN=tablaErrores['FN'].sum()
p= (TP/(TP+FP))
r= (TP/(TP+FN))
F1=2*(p*r)/(p+r)
F1

0.674361088211047

In [24]:
xgb_regC = xgb.XGBRegressor(objective='binary:logistic'\
                           ,colsample_bytree=0.6,learning_rate=0.2,\
                           max_depth=8,alpha=5,n_estimators=200)
xgb_regC.fit(x,y)
preds=xgb_reg.predict(xComp)
preds1 = np.around(preds)
preds1 = preds1.astype(int)
preds1

array([0, 0, 1, ..., 1, 1, 0])

In [25]:
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [26]:
sample_submission['target'] = preds1
sample_submission.to_csv("submission.csv", index=False)
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      3263 non-null   int64
 1   target  3263 non-null   int64
dtypes: int64(2)
memory usage: 51.1 KB


# 